# Understanding patch-clamp data

In this notebook, we look in detail at processing data obtained from patch-clamp experiments.

In particular, we focus on **manual patch clamp** experiments in the **whole cell configuration**.
These can be used to either

1. pass current through the membrane and measure the resulting voltage (_current clamp_); or 
2. control the membrane voltage and measure the resulting current (_voltage clamp_).

We will focus exclusively on **voltage clamp**.

## The set up

Detailed descriptions of the patch clamp technique can be found in the chapter [Penner (1995) A Practical Guide to Patch Clamping](https://doi.org/10.1007/978-1-4419-1229-9_1), the (short) book [Molleman (2002) Patch Clamping](https://doi.org/10.1002/0470856521), or the (slightly older) article [Hamill, Marty et al., Sigworth (1981) Improved patch-clamp techniques for high-resolution current recording from cells and cell-free membrane patches](https://doi.org/10.1007/BF00656997).

In brief, a cell is placed in a bath containing an electrode (the _bath electrode_) and an ion-rich solution that can conduct electricity.
A patch clamp _pipette_ is constructed by pulling on a heated glass tube to form a sharp tip, then filling it with another ion-rich solution, and inserting an electrode into the open end.
The sharp end of the pipette is then placed against the cell membrane and suction, electric pulses, or chemicals are used to break the membrane inside the pipette opening, establishing an electrical connection to the inside of the cell.
A _patch clamp amplifier_ is attached to both electrodes, and can now be used to measure and manipulate the voltage between the bath and pipette electrodes.


## Patch-clamp amplifier electronics

Now that we have access to the inside & outside of the cell, how can we control its voltage and measure the current?

The answer involves a lot of electronics, but a brief outline is given below.
It follows the exposition in [Sigworth 1995a](https://doi.org/10.1007/978-1-4419-1229-9_4), but also uses [Weerakoon et al., 2009](https://doi.org/10.1109/TBCAS.2008.2005419) and [Lei et al., 2020](https://doi.org/10.1098/rsta.2019.0348).
Note that the schematics given here are all simplifications in two ways: 1. They omit a lot of the extra circuitry you'd find in a real world application, and 2. They mostly omit non-ideal effects of components, which can come into play for precise measurements of small currents.

The image below (adapted from Sigworth 1995) shows a very simple schematic to measure small currents.
A battery $V_c$ is attached to a pipette, and a resistor with a known resistance $R$ is introduced.
To obtain the current flowing into the patch clamp assembly, $I_\text{in}$, we measure the voltage drop $V_\text{out} = I_\text{in} R$ across the resistor and use

$$ I_\text{in} = V_\text{out} / R $$

<img src="resources/patch-amp-1-too-simple.png" />

_**Figure 1**: A very simple current measuring device._

By choosing a very large $R$ we get a measurable $V_\text{out}$ for even very small $I_\text{in}$.
However, it also creates a big difference between $V_c$, which we control, and the voltage over the patch clamp assembly.

A clever trick with an [op amp](https://en.wikipedia.org/wiki/Operational_amplifier) can get us around this:

<img src="resources/patch-amp-2-with-op-amp.png" />

_**Figure 2**: A better way to measure small currents._

To analyse this circuit, we use two properties of **an idealised** op-amp:

1. Connected in a negative feedback loop like above, the op amp instantaneously adjusts its output $V_o$ until the voltages at its input terminals ($V_+$ and $V_-$) are the same.
2. No current flows into (or out of) the input terminals.

From the first property we get $V_- = V_c$ and the second property gives us $I_\text{in} = I_R$.
The voltage-drop across the resistor is then

$$ V_o - V_- = V_o - V_c = I_R R = I_\text{in} R $$

And so if we can measure $V_\text{out} \equiv V_o - V_c$ we can use the known value of $R$ to calculate

$$I_\text{in} = V_\text{out} / R$$

Finally, we add a [difference amplifier](https://en.wikipedia.org/wiki/Differential_amplifier) to $V_o$ and $V_c$.
A difference amplifier takes the voltage between its two inputs and multiplies it by a fixed factor, using an external power source.
Here we use an amplification factor of 1 so that it acts as a simple _buffer_.
This means that the power drawn from $V_{out}$ by any connected measurement equipment will be provided by the amplifier's power source instead of the circuit we're trying to measure.

<img src="resources/patch-amp-3-diff-amp.png" />

_**Figure 3**: A difference amplifier buffers $V_{out}$._

For more about op amps and difference amplifiers, see [Appendix A](./appendix-A-op-amp.ipynb).

### Stray capacitance

We now make the schematic a bit more realistic, by adding a capacitor in parallel with the resistance.
Originally, this was done to model "stray" or "parasitic" capacitance of the resistor itself (see e.g. [Sigworth 1995a](https://doi.org/10.1007/978-1-4419-1229-9_4)), but the recent paper by [Weerakoon et al., 2009](https://doi.org/10.1109/TBCAS.2008.2005419) describes it as a separate physical component, "necessary to make the trans-impedance amplifier stable and to increase the bandwidth of the voltage clamp".

<img src="resources/patch-amp-4-Cf.png" />

_**Figure 4**: A stray capacitance exists in parallel with the feedback resistor._

Because the resistance and capacitance are both in the feedback path of the op amp, we will label the resistance and its capacitance with a small f, for "feedback". We can redo the analysis with $C_f$ in place, to see how it affects $V_{out}$. 

Because there are now two pathways for the current to flow through we start from:

\begin{align}
I_\text{in} &= I_R + I_C \\
            &= (V_o - V_-) / R_f + C_f \frac{d}{dt}\left(V_o - V_-\right) \\
            &= V_\text{out} / R_f + C_f \dot{V}_\text{out} \\
V_\text{out} &= I_\text{in} R_f - R_f C_f \dot{V}_\text{out}
\end{align}

To allow for the idea that the current we calculate is no longer equal to $I_\text{in}$, we introduce a new symbol

\begin{align}
I_\text{obs} \equiv V_\text{out} / R_f
\end{align}

We can rearrange and differentiate to get $V_\text{out} = R_f I_\text{obs}$ and $\dot{V}_\text{out} = R_f \dot{I}_\text{obs}$, so that

\begin{align}
V_\text{out} &= I_\text{in} R_f - R_f C_f \dot{V}_\text{out}
\end{align}
divide by $R_f$ and rearrange to find
\begin{align}
I_\text{obs} = I_\text{in} - R_f C_f \dot{I}_\text{obs}
\end{align}
and
\begin{align}
\dot{I}_\text{obs} &= \frac{I_\text{in} - I_\text{obs}}{R_f C_f}
                   = \frac{I_\text{in} - I_\text{obs}}{\tau_f}
\end{align}

This means that $I_\text{obs}$ will grow when it's smaller than $I_\text{in}$ and shrink when it's larger: in other words $I_\text{obs}$ will follow $I_\text{in}$ with a time constant $\tau_f = R_f C_f$.
Typical values for $R_f$ and $C_f$ are given in [Appendix B](./appendix-B-Rf-and-Cf.ipynb).
For whole-cell experiments with standard gain settings, you might expect a $\tau_f$ on the order of $80\,{\mu}s$ (HEKA) to $500\,{\mu}s$ (Axon).

### More capacitance!

We now extend our diagram with another capacitor, $C_p$, to represent the "pipette capacitance".

In manual patch clamp, the submerged part of the pipette will act as a capacitor (for a detailed description, see [Levis & Rae 1998](https://doi.org/10.1016/S0076-6879(98)93017-8)).
The value of $C_p$ depends on the tip shape and wall thickness of the (home made) pipette, and on how much of it is immersed.
As a result, $C_p$ will vary between experiments.
During an experiment, changes in the water level due to evaporation and/or perfusion as well as adhesion effects (water slowly creeping up the side of the pipette) can cause further variation (see e.g. [Thompson et al. 2001](https://doi.org/10.1016/S0006-3495(01)75752-9)).

However, there are other capacitative effects in the set up, and a considerable $C_p$ exists in pipette-free patch clamp methods such as planar patch.
So it is better to think of $C_p$ as a lumped, _parasitic_ capacitance.

<img src="resources/patch-amp-5-Cp.png" />

_**Figure 5**: The pipette, or lumped parasitic capacitance._

In the diagram above, we have added in $C_p$, and renamed the voltage $V_-$ to $V_p$, as we will use $V_p$ and $C_p$ together in the equations.

Using the temporary symbol $I$ for the current flowing out on the left, we can then write
\begin{align}
I_{in} = I + C_p \dot{V}_p
\end{align}
and
\begin{align}
\dot{I}_\text{obs} = \frac{I + C_p \dot{V}_p - I_\text{obs}}{\tau_f}
\end{align}


Using the ideal op-amp assumptions, we have $V_p = V_c$ and $\dot{V}_p = \dot{V}_c$, so the new term is entirely dependent on our input signal for $V_c$.
For a step protocol (a very common choice in voltage clamping), $\dot{V}_p$ is either 0 during the steps or infinity at the discontinuities, which suggests that our idealised assumptions are reaching the limits of their usefulness.

### A less idealised op amp

We now revisit the ideal op amp assumptions:

1. We will keep the assumption that no current flows into the terminals
2. But we give the op amp a finite speed: for it's output voltage $V_o$ we will write $\dot{V}_o = \left(V_+ - V_-\right) / \tau_\text{amp}$

An estimate for $\tau_\text{amp}$ from Weerakoon et al. is 0.05 $\mu$s, while Sigworth 1995a gives an example value of 0.1 $\mu$s.